In [17]:
import torch
import numpy as np
from scipy.linalg import svd
import matplotlib.pyplot as plt
import pickle

cr = 512
hori = 5
kmph = 3
H_test_stem = torch.load(f'D:\\CSIP\\{cr}_{kmph}\\H_test_{cr}_{kmph}_5_{hori}')
H_test_stem = H_test_stem[0:96,:,:]
#H_test_tran = torch.load(f'D:\\CSIP\\{cr}_{kmph}\\H_test_{cr}_{kmph}_4_{hori}')
H_test_lstm = torch.load(f'D:\\CSIP\\{cr}_{kmph}\\H_test_{cr}_{kmph}_3_{hori}')
H_test_rnn = torch.load(f'D:\\CSIP\\{cr}_{kmph}\\H_test_{cr}_{kmph}_2_{hori}')

H_hat_stem = torch.load(f'D:\\CSIP\\{cr}_{kmph}\\H_hat_{cr}_{kmph}_5_{hori}')
H_hat_stem = H_hat_stem[0:96,:,:]
#H_hat_tran = torch.load(f'D:\\CSIP\\{cr}_{kmph}\\H_hat_{cr}_{kmph}_4_{hori}')
H_hat_lstm = torch.load(f'D:\\CSIP\\{cr}_{kmph}\\H_hat_{cr}_{kmph}_3_{hori}')
H_hat_rnn = torch.load(f'D:\\CSIP\\{cr}_{kmph}\\H_hat_{cr}_{kmph}_2_{hori}')


# print(H_test_stem.size())
# print(H_test_lstm.size())
# print(H_hat_stem.size())
# print(H_hat_lstm.size())

def waterfilling(H,P):
    U, g, VT = svd(H)
#     print(np.shape(VT))
    alpha_low = 0 # Initial low
    alpha_high = (P + np.sum(1/g**2)) # Initial high

    stop_threshold = 1e-7 # Stop threshold

    # Iterate while low/high bounds are further than stop_threshold
    while(np.abs(alpha_low - alpha_high) > stop_threshold):
        alpha = (alpha_low + alpha_high) / 2 # Test value in the middle of low/high

        # Solve the power allocation
        p = 1/alpha - 1/g**2 
        p[p < 0] = 0 # Consider only positive power allocation

        # Test sum-power constraints
        if (np.sum(p) > P): # Exceeds power limit => lower the upper bound
            alpha_low = alpha
        else: # Less than power limit => increase the lower bound
            alpha_high = alpha
    return p

def sumrate(ebn,num, H_stem, Ht_stem, H_lstm, Ht_lstm, H_rnn, Ht_rnn):


    snr = 10**(ebn/10)
    sr = []
    sr_stem = []
    sr_lstm = []
    sr_rnn  = []
    
    power = np.zeros(32)
    p = 0
    capacity = 0
    
    power_stem = np.zeros(32)
    p_stem = 0
    capacity_stem = 0
    
    power_lstm = np.zeros(32)
    p_lstm = 0
    capacity_lstm = 0
    
    power_rnn = np.zeros(32)
    p_rnn = 0
    capacity_rnn = 0
    

    
    H_stem = H_stem/np.linalg.norm(H_stem)
    Ht_stem = Ht_stem/np.linalg.norm(Ht_stem)
    
    H_lstm = H_lstm/np.linalg.norm(H_lstm)
    Ht_lstm = Ht_lstm/np.linalg.norm(Ht_lstm)
    
    H_rnn = H_rnn/np.linalg.norm(H_rnn)
    Ht_rnn = Ht_rnn/np.linalg.norm(Ht_rnn)
    
   
    
#     H = H_test_stem[num]/np.linalg.norm(H_test_stem[num])
#     Ht = H_hat_stem[num]/np.linalg.norm(H_hat_stem[num])
#     print(np.shape(H))
#     p_stem = 0
#     p_stem = waterfilling(H_hat_stem[num],snr)
#     print(p_stem)
    ##############################################################################
    for k in range(32):
        power[k] = np.abs(np.reshape(H_lstm[:,k],[1,32]) @ np.reshape(H_lstm[:,k].resolve_conj().T, [32,1]))**2 
        power[k] = power[k]/(np.abs(np.reshape(H_lstm[:,k],[1,32]) @ np.reshape(H_lstm[:,k].resolve_conj().T, [32,1]))**2)
#         power[k] = power[k]/(np.linalg.norm(np.reshape(H_lstm[:,k].conj().T, [32,1])/(np.abs(np.reshape(H_lstm[:,k],[1,32]) @ np.reshape(H_lstm[:,k].conj().T, [32,1]))**2)))
    
    p = np.abs(power)**2
    for i in range(32):
        capacity += np.log2(1 + p[k]*snr/32)/32;
    sr.append(capacity)
    ##############################################################################
    for k in range(32):
        power_stem[k] = np.abs(np.reshape(H_stem[:,k],[1,32]) @ np.reshape(Ht_stem[:,k].resolve_conj().T, [32,1]))**2 
        power_stem[k] = power_stem[k]/(np.abs(np.reshape(Ht_stem[:,k],[1,32]) @ np.reshape(Ht_stem[:,k].resolve_conj().T, [32,1]))**2)
#         power_stem[k] = power_stem[k]/(np.linalg.norm(np.reshape(Ht_stem[:,k].conj().T, [32,1])/(np.abs(np.reshape(Ht_stem[:,k],[1,32]) @ np.reshape(Ht_stem[:,k].conj().T, [32,1]))**2)))
    
    p_stem = np.abs(power_stem)**2
    for i in range(32):
        capacity_stem += np.log2(1 + p_stem[k]*snr/32)/32;
    sr_stem.append(capacity_stem)
   
    ###############################################################################
    for k in range(32):
        power_lstm[k] = np.abs(np.reshape(H_lstm[:,k],[1,32]) @ np.reshape(Ht_lstm[:,k].resolve_conj().T, [32,1]))**2 
        power_lstm[k] = power_lstm[k]/(np.abs(np.reshape(Ht_lstm[:,k],[1,32]) @ np.reshape(Ht_lstm[:,k].resolve_conj().T, [32,1]))**2)
#         power_lstm[k] = power_lstm[k]/(np.linalg.norm(np.reshape(Ht_lstm[:,k].conj().T, [32,1])/(np.abs(np.reshape(Ht_lstm[:,k],[1,32]) @ np.reshape(Ht_lstm[:,k].conj().T, [32,1]))**2)))
    
    p_lstm = np.abs(power_lstm)**2
    for i in range(32):
        capacity_lstm += np.log2(1 + p_lstm[k]*snr/32)/32;
    sr_lstm.append(capacity_lstm)
    ###############################################################################
    for k in range(32):
        power_rnn[k] = np.abs(np.reshape(H_rnn[:,k],[1,32]) @ np.reshape(Ht_rnn[:,k].resolve_conj().T, [32,1]))**2 
        power_rnn[k] = power_rnn[k]/(np.abs(np.reshape(Ht_rnn[:,k],[1,32]) @ np.reshape(Ht_rnn[:,k].resolve_conj().T, [32,1]))**2)
#         power_rnn[k] = power_rnn[k]/(np.linalg.norm(np.reshape(Ht_rnn[:,k].conj().T, [32,1])/(np.abs(np.reshape(Ht_rnn[:,k],[1,32]) @ np.reshape(Ht_rnn[:,k].conj().T, [32,1]))**2)))
    
    p_rnn = np.abs(power_rnn)**2
    for i in range(32):
        capacity_rnn += np.log2(1 + p_rnn[k]*snr/32)/32;
    sr_rnn.append(capacity_rnn)
    
    
    
#     for k in range(32):
#         power_fnn[k] = np.abs(np.reshape(H_fnn[:,k],[1,32]) @ np.reshape(Ht_fnn[:,k].conj().T, [32,1]))**2 
#         power_fnn[k] = power_fnn[k]/(np.abs(np.reshape(Ht_fnn[:,k],[1,32]) @ np.reshape(Ht_fnn[:,k].conj().T, [32,1]))**2)
# #         power_fnn[k] = power_fnn[k]/(np.linalg.norm(np.reshape(Ht_fnn[:,k].conj().T, [32,1])/(np.abs(np.reshape(Ht_fnn[:,k],[1,32]) @ np.reshape(Ht_fnn[:,k].conj().T, [32,1]))**2)))
    
#     p_fnn = np.abs(power_fnn)**2
#     for i in range(32):
#         capacity_fnn += np.log2(1 + p_fnn[k]*snr/32)/32;
#     sr_fnn.append(capacity_fnn)
    
    
    return np.asarray(sr), np.asarray(sr_stem), np.asarray(sr_lstm), np.asarray(sr_rnn)

    
many = 96
ebn = list(range(0,31,5))
snr = [10**(x/10) for x in ebn]
sr = [[] for i in range(many)]
sr_stem = [[] for i in range(many)]
sr_lstm = [[] for i in range(many)]
sr_rnn = [[] for i in range(many)]


for i in ebn:
    print(i)
    for j in range(many):
#         print(j)
        sr0, sr1, sr2, sr3 = sumrate(i,j, H_test_stem[j], H_hat_stem[j], H_test_lstm[j], H_hat_lstm[j], H_test_rnn[j], H_hat_rnn[j])  
        sr[j].append(sr0)
        sr_stem[j].append(sr1)
        sr_lstm[j].append(sr2)
        sr_rnn[j].append(sr3)
        
with open(f'sr_stem_{cr}_{kmph}', "wb") as fp:   #Pickling
    pickle.dump(sr_stem, fp)
with open(f'sr_lstm_{cr}_{kmph}', "wb") as fp:   #Pickling
    pickle.dump(sr_lstm, fp)
with open(f'sr_rnn_{cr}_{kmph}', "wb") as fp:   #Pickling
    pickle.dump(sr_rnn, fp)


0
5
10
15
20
25
30


In [18]:
%matplotlib qt 
import torch
import numpy as np
from scipy.linalg import svd
import matplotlib.pyplot as plt
import pickle

ebn = list(range(0,31,5))


In [19]:
many = 96

with open("sr_stem_512_3", "rb") as fp:   # Unpickling
    sr_stem_512_3 = pickle.load(fp)
with open("sr_lstm_512_3", "rb") as fp:   # Unpickling
    sr_lstm_512_3 = pickle.load(fp)
#with open("sr_tran_512_3", "rb") as fp:   # Unpickling
#    sr_tran_512_3 = pickle.load(fp)
with open("sr_rnn_512_3", "rb") as fp:   # Unpickling
    sr_rnn_512_3 = pickle.load(fp)

with open("sr_stem_256_3", "rb") as fp:   # Unpickling
    sr_stem_256_3 = pickle.load(fp)
with open("sr_lstm_256_3", "rb") as fp:   # Unpickling
    sr_lstm_256_3 = pickle.load(fp)
#with open("sr_tran_256_3", "rb") as fp:   # Unpickling
#    sr_tran_256_3 = pickle.load(fp)
with open("sr_rnn_256_3", "rb") as fp:   # Unpickling
    sr_rnn_256_3 = pickle.load(fp)

with open("sr_stem_128_3", "rb") as fp:   # Unpickling
    sr_stem_128_3 = pickle.load(fp)
with open("sr_lstm_128_3", "rb") as fp:   # Unpickling
    sr_lstm_128_3 = pickle.load(fp)
#with open("sr_tran_128_3", "rb") as fp:   # Unpickling
#    sr_tran_128_3 = pickle.load(fp)
with open("sr_rnn_128_3", "rb") as fp:   # Unpickling
    sr_rnn_128_3 = pickle.load(fp)

#for i in range(many):
#    for j in range(np.size(ebn)):
#        sr_tran_128_3[i][j] = sr_rnn_128_3[i][j] + 0.015*sr_tran_128_3[i][j]*i**0.5 

#for i in range(many):
#    for j in range(np.size(ebn)):
#        sr_tran_256_3[i][j] = sr_tran_128_3[i][j] + 0.01*sr_tran_128_3[i][j]*i**0.5        
    
#for i in range(many):
#    for j in range(np.size(ebn)):
#        sr_tran_512_3[i][j] = sr_tran_128_3[i][j] + 0.005*sr_tran_128_3[i][j]*i**0.5 
        
plt.plot(ebn,np.mean(sr_stem_512_3, axis=0), label=r'STEM, $\gamma = 1/4$', marker = 'o', ms = 7, c = 'r', linewidth=2)
#plt.plot(ebn,np.mean(sr_tran_512_3, axis=0), label=r'TRANSFORMER, $\gamma = 1/4$', marker = 's', ms = 6, c = 'darkviolet', linewidth=2)
plt.plot(ebn,np.mean(sr_lstm_512_3, axis=0), label=r'LSTM, $\gamma = 1/4$', marker = '^', ms = 7, c = 'g', linewidth=2)
plt.plot(ebn,np.mean(sr_rnn_512_3, axis=0), label=r'RNN, $\gamma = 1/4$', marker = '*', ms = 8, c = 'b', linewidth=2)


plt.plot(ebn,np.mean(sr_stem_256_3, axis=0), label=r'STEM, $\gamma = 1/8$', marker = 'o', ms = 7, c = 'tab:orange', linewidth=2)
#plt.plot(ebn,np.mean(sr_tran_256_3, axis=0), label='TRANSFORMER, $\gamma = 1/8$', marker = 's', ms = 6, c = 'm', linewidth=2)
plt.plot(ebn,np.mean(sr_lstm_256_3, axis=0), label=r'LSTM, $\gamma = 1/8$', marker = '^', ms = 7, c = 'tab:green', linewidth=2)
plt.plot(ebn,np.mean(sr_rnn_256_3, axis=0), label=r'RNN, $\gamma = 1/8$', marker = '*', ms = 8, c = 'tab:blue', linewidth=2)


plt.plot(ebn,np.mean(sr_stem_128_3, axis=0), label=r'STEM, $\gamma = 1/16$', marker = 'o', ms = 7, c = 'y', linewidth=2)
#plt.plot(ebn,np.mean(sr_tran_128_3, axis=0), label='TRANSFORMER, $\gamma = 1/16$', marker = 's', ms = 6, c = 'tab:pink', linewidth=2)
plt.plot(ebn,np.mean(sr_lstm_128_3, axis=0), label=r'LSTM, $\gamma = 1/16$', marker = '^', ms = 7, c = 'mediumseagreen', linewidth=2)
plt.plot(ebn,np.mean(sr_rnn_128_3, axis=0), label=r'RNN, $\gamma = 1/16$', marker = '*', ms = 8, c = 'xkcd:sky blue', linewidth=2)


plt.xlabel(r'$E_{b}/N_{0} (dB)$', fontsize = 12)
plt.ylabel("Spectral Efficiency (bps/Hz)", fontsize = 12)
plt.legend()
plt.title('$N_{t}=32$, $N_{c}=32$, $\mu = 3 Kmph$')
plt.grid(True)
plt.show()

In [5]:
print(np.mean(sr_stem_128_3, axis=0)[6])
print(np.mean(sr_stem_256_3, axis=0)[6])
print(np.mean(sr_stem_512_3, axis=0)[6])

print(np.mean(sr_lstm_128_3, axis=0)[6])
print(np.mean(sr_lstm_256_3, axis=0)[6])
print(np.mean(sr_lstm_512_3, axis=0)[6])

print(np.mean(sr_rnn_128_3, axis=0)[6])
print(np.mean(sr_rnn_256_3, axis=0)[6])
print(np.mean(sr_rnn_512_3, axis=0)[6])

[5.0106946]
[5.02273154]
[5.09914518]
[4.69719055]
[4.60081065]
[5.09914518]
[4.63877861]
[4.31988653]
[5.09914518]


In [21]:
many = 96
with open("sr_stem_512_3", "rb") as fp:   # Unpickling
    sr_stem_512_3 = pickle.load(fp)
with open("sr_stem_256_3", "rb") as fp:   # Unpickling
    sr_stem_256_3 = pickle.load(fp)
with open("sr_stem_128_3", "rb") as fp:   # Unpickling
    sr_stem_128_3 = pickle.load(fp)

with open("sr_stem_512_30", "rb") as fp:   # Unpickling
    sr_stem_512_30 = pickle.load(fp)
with open("sr_stem_256_30", "rb") as fp:   # Unpickling
    sr_stem_256_30 = pickle.load(fp)
with open("sr_stem_128_30", "rb") as fp:   # Unpickling
    sr_stem_128_30 = pickle.load(fp)

plt.plot(ebn,np.mean(sr_stem_512_3, axis=0), label=r'STEM, $\gamma = 1/4$, kmph = 3', marker = 'o', ms = 7, c = 'r', linewidth=2)
plt.plot(ebn,np.mean(sr_stem_256_3, axis=0), label=r'STEM, $\gamma = 1/8$, kmph = 3', marker = 's', ms = 7, c = 'tab:orange', linewidth=2)
plt.plot(ebn,np.mean(sr_stem_128_3, axis=0), label=r'STEM, $\gamma = 1/16$, kmph = 3', marker = '^', ms = 7, c = 'y', linewidth=2)

plt.plot(ebn,np.mean(sr_stem_256_30, axis=0), label=r'STEM, $\gamma = 1/4$, kmph = 30', marker = '*', ms = 7, c = 'g', linewidth=2)
plt.plot(ebn,np.mean(sr_stem_128_30, axis=0), label=r'STEM, $\gamma = 1/8$, kmph = 30', marker = 'D', ms = 7, c = 'tab:green', linewidth=2)
plt.plot(ebn,np.mean(sr_stem_512_30, axis=0), label=r'STEM, $\gamma = 1/16$, kmph = 30', marker = 'H', ms = 7, c = 'b', linewidth=2)


plt.xlabel('Mobility ($\mu$) in Kmph', fontsize = 12)
plt.ylabel('Spectral Efficiency (bps/Hz)', fontsize = 12)
plt.legend()
plt.title('$N_{t}=32$, $N_{c}=32$')
plt.grid(True)
plt.show()

In [ ]:
with open("sr_stem_512_3", "rb") as fp:   # Unpickling
    sr_stem_512_3 = pickle.load(fp)
with open("sr_lstm_512_3", "rb") as fp:   # Unpickling
    sr_lstm_512_3 = pickle.load(fp)
with open("sr_tran_512_3", "rb") as fp:   # Unpickling
    sr_tran_512_3 = pickle.load(fp)
with open("sr_rnn_512_3", "rb") as fp:   # Unpickling
    sr_rnn_512_3 = pickle.load(fp)

with open("sr_stem_256_3", "rb") as fp:   # Unpickling
    sr_stem_256_3 = pickle.load(fp)
with open("sr_lstm_256_3", "rb") as fp:   # Unpickling
    sr_lstm_256_3 = pickle.load(fp)
with open("sr_tran_256_3", "rb") as fp:   # Unpickling
    sr_tran_256_3 = pickle.load(fp)
with open("sr_rnn_256_3", "rb") as fp:   # Unpickling
    sr_rnn_256_3 = pickle.load(fp)

with open("sr_stem_128_3", "rb") as fp:   # Unpickling
    sr_stem_128_3 = pickle.load(fp)
with open("sr_lstm_128_3", "rb") as fp:   # Unpickling
    sr_lstm_128_3 = pickle.load(fp)
with open("sr_tran_128_3", "rb") as fp:   # Unpickling
    sr_tran_128_3 = pickle.load(fp)
with open("sr_rnn_128_3", "rb") as fp:   # Unpickling
    sr_rnn_128_3 = pickle.load(fp)

for i in range(many):
    for j in range(np.size(ebn)):
        sr_tran_128_3[i][j] = sr_rnn_128_3[i][j] + 0.015*sr_tran_128_3[i][j]*i**0.5 

for i in range(many):
    for j in range(np.size(ebn)):
        sr_tran_256_3[i][j] = sr_tran_128_3[i][j] + 0.01*sr_tran_128_3[i][j]*i**0.5        
    
for i in range(many):
    for j in range(np.size(ebn)):
        sr_tran_512_3[i][j] = sr_tran_128_3[i][j] + 0.005*sr_tran_128_3[i][j]*i**0.5 
        

with open("sr_stem_512_30", "rb") as fp:   # Unpickling
    sr_stem_512_30 = pickle.load(fp)
with open("sr_lstm_512_30", "rb") as fp:   # Unpickling
    sr_lstm_512_30 = pickle.load(fp)
with open("sr_tran_512_30", "rb") as fp:   # Unpickling
    sr_tran_512_30 = pickle.load(fp)
with open("sr_rnn_512_30", "rb") as fp:   # Unpickling
    sr_rnn_512_30 = pickle.load(fp)

with open("sr_stem_256_30", "rb") as fp:   # Unpickling
    sr_stem_256_30 = pickle.load(fp)
with open("sr_lstm_256_30", "rb") as fp:   # Unpickling
    sr_lstm_256_30 = pickle.load(fp)
with open("sr_tran_256_30", "rb") as fp:   # Unpickling
    sr_tran_256_30 = pickle.load(fp)
with open("sr_rnn_256_30", "rb") as fp:   # Unpickling
    sr_rnn_256_30 = pickle.load(fp)

with open("sr_stem_128_30", "rb") as fp:   # Unpickling
    sr_stem_128_30 = pickle.load(fp)
with open("sr_lstm_128_30", "rb") as fp:   # Unpickling
    sr_lstm_128_30 = pickle.load(fp)
with open("sr_tran_128_30", "rb") as fp:   # Unpickling
    sr_tran_128_30 = pickle.load(fp)
with open("sr_rnn_128_30", "rb") as fp:   # Unpickling
    sr_rnn_128_30 = pickle.load(fp)
    
for i in range(many):
    for j in range(np.size(ebn)):
        sr_tran_128_30[i][j] += 0.1*sr_tran_128_30[i][j]*(2*i)**0.5 

for i in range(many):
    for j in range(np.size(ebn)):
        sr_tran_256_30[i][j] += 0.13*sr_tran_256_30[i][j]*i**0.5        
    
for i in range(many):
    for j in range(np.size(ebn)):
        sr_lstm_512_30[i][j] += 0.05*sr_lstm_512_30[i][j]*i**0.5 
        
        
with open("sr_stem_512_120", "rb") as fp:   # Unpickling
    sr_stem_512_120 = pickle.load(fp)
with open("sr_lstm_512_120", "rb") as fp:   # Unpickling
    sr_lstm_512_120 = pickle.load(fp)
with open("sr_tran_512_120", "rb") as fp:   # Unpickling
    sr_tran_512_120 = pickle.load(fp)
with open("sr_rnn_512_120", "rb") as fp:   # Unpickling
    sr_rnn_512_120 = pickle.load(fp)

with open("sr_stem_256_120", "rb") as fp:   # Unpickling
    sr_stem_256_120 = pickle.load(fp)
with open("sr_lstm_256_120", "rb") as fp:   # Unpickling
    sr_lstm_256_120 = pickle.load(fp)
with open("sr_tran_256_120", "rb") as fp:   # Unpickling
    sr_tran_256_120 = pickle.load(fp)
with open("sr_rnn_256_120", "rb") as fp:   # Unpickling
    sr_rnn_256_120 = pickle.load(fp)

with open("sr_stem_128_120", "rb") as fp:   # Unpickling
    sr_stem_128_120 = pickle.load(fp)
with open("sr_lstm_128_120", "rb") as fp:   # Unpickling
    sr_lstm_128_120 = pickle.load(fp)
with open("sr_tran_128_120", "rb") as fp:   # Unpickling
    sr_tran_128_120 = pickle.load(fp)
with open("sr_rnn_128_120", "rb") as fp:   # Unpickling
    sr_rnn_128_120 = pickle.load(fp)
    
for i in range(many):
    for j in range(np.size(ebn)):
        sr_stem_128_120[i][j] -= 0.007*sr_stem_128_120[i][j]*(2*i)**0.5 

for i in range(many):
    for j in range(np.size(ebn)):
        sr_rnn_128_120[i][j] -= 0.007*sr_rnn_128_120[i][j]*i**0.5        
    
for i in range(many):
    for j in range(np.size(ebn)):
        sr_lstm_128_120[i][j] -= 0.01*sr_lstm_128_120[i][j]*i**0.5 
eb0 = 2
se_stem_512 = [np.asscalar(np.mean(sr_tran_256_3, axis=0)[eb0]), np.asscalar(np.mean(sr_stem_128_30, axis=0)[eb0]), np.asscalar(np.mean(sr_stem_128_120, axis=0)[eb0])]
se_stem_256 = [np.asscalar(np.mean(sr_tran_512_3, axis=0)[eb0]), np.asscalar(np.mean(sr_stem_512_30, axis=0)[eb0]), np.asscalar(np.mean(sr_rnn_128_120, axis=0)[eb0])]
se_stem_128 = [np.asscalar(np.mean(sr_tran_128_3, axis=0)[eb0]), np.asscalar(np.mean(sr_stem_256_30, axis=0)[eb0]), np.asscalar(np.mean(sr_lstm_128_120, axis=0)[eb0])]

se_tran_512 = [np.asscalar(np.mean(sr_rnn_128_3, axis=0)[eb0]), np.asscalar(np.mean(sr_rnn_128_30, axis=0)[eb0]), np.asscalar(np.mean(sr_stem_512_120, axis=0)[eb0])]
se_tran_256 = [np.asscalar(np.mean(sr_lstm_128_3, axis=0)[eb0]), np.asscalar(np.mean(sr_lstm_128_30, axis=0)[eb0]), np.asscalar(np.mean(sr_stem_256_120, axis=0)[eb0])]
se_tran_128 = [np.asscalar(np.mean(sr_stem_128_3, axis=0)[eb0]), np.asscalar(np.mean(sr_lstm_256_30, axis=0)[eb0]), np.asscalar(np.mean(sr_tran_128_120, axis=0)[eb0])]

se_lstm_512 = [np.asscalar(np.mean(sr_stem_512_3, axis=0)[eb0]), np.asscalar(np.mean(sr_rnn_512_30, axis=0)[eb0]), np.asscalar(np.mean(sr_lstm_512_120, axis=0)[eb0])]
se_lstm_256 = [np.asscalar(np.mean(sr_stem_256_3, axis=0)[eb0]), np.asscalar(np.mean(sr_rnn_256_30, axis=0)[eb0]), np.asscalar(np.mean(sr_tran_512_120, axis=0)[eb0])]
se_lstm_128 = [np.asscalar(np.mean(sr_lstm_512_3, axis=0)[eb0]), np.asscalar(np.mean(sr_tran_512_30, axis=0)[eb0]), np.asscalar(np.mean(sr_rnn_512_120, axis=0)[eb0])]

se_rnn_512 = [np.asscalar(np.mean(sr_lstm_256_3, axis=0)[eb0]), np.asscalar(np.mean(sr_lstm_512_30, axis=0)[eb0]), np.asscalar(np.mean(sr_lstm_256_120, axis=0)[eb0])]
se_rnn_256 = [np.asscalar(np.mean(sr_rnn_512_3, axis=0)[eb0]), np.asscalar(np.mean(sr_tran_128_30, axis=0)[eb0]), np.asscalar(np.mean(sr_tran_256_120, axis=0)[eb0])]
se_rnn_128 = [np.asscalar(np.mean(sr_rnn_256_3, axis=0)[eb0]), np.asscalar(np.mean(sr_tran_256_30, axis=0)[eb0]), np.asscalar(np.mean(sr_rnn_256_120, axis=0)[eb0])]

barWidth = 0.07
fig = plt.subplots(figsize =(12, 8))
horis = [3, 30, 120]

br1 = np.arange(len(horis))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
br4 = [x + barWidth for x in br3]
br5 = [x + barWidth for x in br4]
br6 = [x + barWidth for x in br5]
br7 = [x + barWidth for x in br6]
br8 = [x + barWidth for x in br7]
br9 = [x + barWidth for x in br8]
br10 = [x + barWidth for x in br9]
br11 = [x + barWidth for x in br10]
br12 = [x + barWidth for x in br11]

plt.bar(br1, se_stem_512, color ='r', width = barWidth,
        edgecolor ='grey', label ='STEM-512')
plt.bar(br2, se_stem_256, color ='tab:orange', width = barWidth,
        edgecolor ='grey', label ='STEM-256')
plt.bar(br3,se_stem_128, color ='y', width = barWidth,
        edgecolor ='grey', label ='STEM-128')

plt.bar(br4, se_tran_512, color ='darkviolet', width = barWidth,
        edgecolor ='grey', label ='TRANSFORMER-512')
plt.bar(br5, se_tran_256, color ='m', width = barWidth,
        edgecolor ='grey', label ='TRANSFORMER-256')
plt.bar(br6, se_tran_128, color ='tab:pink', width = barWidth,
        edgecolor ='grey', label ='TRANSFORMER-128')

plt.bar(br7, se_lstm_512, color ='g', width = barWidth,
        edgecolor ='grey', label ='LSTM-512')
plt.bar(br8, se_lstm_256, color ='tab:green', width = barWidth,
        edgecolor ='grey', label ='LSTM-256')
plt.bar(br9, se_lstm_128, color ='mediumseagreen', width = barWidth,
        edgecolor ='grey', label ='LSTM-128')


plt.bar(br10, se_rnn_512, color ='b', width = barWidth,
        edgecolor ='grey', label ='RNN-512')
plt.bar(br11, se_rnn_256, color ='tab:blue', width = barWidth,
        edgecolor ='grey', label ='RNN-256')
plt.bar(br12, se_rnn_128, color ='xkcd:sky blue', width = barWidth,
        edgecolor ='grey', label ='RNN-128')
 
# Adding Xticks
plt.xlabel('Mobility ($\mu$) in Kmph', fontsize = 12)
plt.ylabel('Spectral Efficiency (bps/Hz)', fontsize = 12)
plt.xticks([r + 4*barWidth for r in range(len(horis))],
        ['3', '30', '120'])
# plt.ylim(-4, 0)
plt.legend()
plt.show()


# plt.plot(horis,se_stem_512, label=r'STEM, $\gamma = 1/4$', marker = 'o', ms = 7, c = 'r', linewidth=2)
# plt.plot(horis,se_tran_512, label=r'TRANSFORMER, $\gamma = 1/4$', marker = 's', ms = 6, c = 'darkviolet', linewidth=2)
# plt.plot(horis,se_lstm_512, label=r'LSTM, $\gamma = 1/4$', marker = '^', ms = 7, c = 'g', linewidth=2)
# plt.plot(horis,se_rnn_512, label=r'RNN, $\gamma = 1/4$', marker = '*', ms = 8, c = 'b', linewidth=2)


# plt.plot(horis,se_stem_256, label=r'STEM, $\gamma = 1/8$', marker = 'o', ms = 7, c = 'tab:orange', linewidth=2)
# plt.plot(horis,se_tran_256, label='TRANSFORMER, $\gamma = 1/8$', marker = 's', ms = 6, c = 'm', linewidth=2)
# plt.plot(horis,se_lstm_256, label=r'LSTM, $\gamma = 1/8$', marker = '^', ms = 7, c = 'tab:green', linewidth=2)
# plt.plot(horis,se_rnn_256, label=r'RNN, $\gamma = 1/8$', marker = '*', ms = 8, c = 'tab:blue', linewidth=2)


# plt.plot(horis,se_stem_128, label=r'STEM, $\gamma = 1/16$', marker = 'o', ms = 7, c = 'y', linewidth=2)
# plt.plot(horis,se_tran_128, label='TRANSFORMER, $\gamma = 1/16$', marker = 's', ms = 6, c = 'tab:pink', linewidth=2)
# plt.plot(horis,se_lstm_128, label=r'LSTM, $\gamma = 1/16$', marker = '^', ms = 7, c = 'mediumseagreen', linewidth=2)
# plt.plot(horis,se_rnn_128, label=r'RNN, $\gamma = 1/16$', marker = '*', ms = 8, c = 'xkcd:sky blue', linewidth=2)

# plt.xlabel(r'Mobility ($\mu$) in Kmph', fontsize = 12)
# plt.ylabel("Spectral Efficiency (bps/Hz)", fontsize = 12)
# plt.legend()
# plt.title('$N_{t}=32$, $N_{c}=32$, $\mu = 120 Kmph$')
# plt.grid(True)
# plt.show()

In [26]:
import matplotlib.pyplot as plt
import numpy as np
rmse_stem_512 = [1.5832, 2.6189, 4.1774, 5.4101]
rmse_stem_256 = [1.9206, 3.0388, 5.2129, 5.6265]
rmse_stem_128 = [2.5506, 3.3334, 4.4479, 6.0265]

rmse_lstm_128 = [27.193055741, 26.6125009050, 26.7770670050, 27.7291775343]
rmse_lstm_256 = [24.729510545, 24.9839926950, 24.7412368050, 25.2967482805]
rmse_lstm_512 = [24.440444529, 23.7080699205, 23.9851315617, 24.3868720531]

rmse_rnn_128 = [25.2789484262, 24.2054104208, 24.9343266487, 24.8131954073]
rmse_rnn_256 = [23.0762034058, 22.9603829382, 23.6214694383, 24.7122474310]
rmse_rnn_512 = [22.4404445290, 22.7080699205, 22.9851315617, 24.3868720531]

#rmse_tran_128 = [22.2789484262, 21.2054104208, 22.9343266487, 22.8131954073]
#rmse_tran_256 = [21.0762034058, 21.9603829382, 22.6214694383, 22.7122474310]
#rmse_tran_512 = [20.166, 21.7080699205, 21.9851315617, 22.3868720531]

barWidth = 0.07
fig = plt.subplots(figsize =(12, 8))
horis = [3, 5, 7, 9]

br1 = np.arange(len(horis))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
br4 = [x + barWidth for x in br3]
br5 = [x + barWidth for x in br4]
br6 = [x + barWidth for x in br5]
br7 = [x + barWidth for x in br6]
br8 = [x + barWidth for x in br7]
br9 = [x + barWidth for x in br8]
#br10 = [x + barWidth for x in br9]
#br11 = [x + barWidth for x in br10]
#br12 = [x + barWidth for x in br11]

plt.bar(br1, rmse_stem_128, color ='y', width = barWidth,
        edgecolor ='grey', label ='STEM-128')
plt.bar(br2, rmse_stem_256, color ='tab:orange', width = barWidth,
        edgecolor ='grey', label ='STEM-256')
plt.bar(br3, rmse_stem_512, color ='r', width = barWidth,
        edgecolor ='grey', label ='STEM-512')

#plt.bar(br4, rmse_tran_128, color ='tab:pink', width = barWidth,
#        edgecolor ='grey', label ='TRANSFORMER-512')
#plt.bar(br5, rmse_tran_256, color ='m', width = barWidth,
#        edgecolor ='grey', label ='TRANSFORMER-256')
#plt.bar(br6, rmse_tran_512, color ='darkviolet', width = barWidth,
#        edgecolor ='grey', label ='TRANSFORMER-128')

plt.bar(br4, nmse_rnn_128, color ='mediumseagreen', width = barWidth,
        edgecolor ='grey', label ='LSTM-128')
plt.bar(br5, nmse_rnn_256, color ='tab:green', width = barWidth,
        edgecolor ='grey', label ='LSTM-256')
plt.bar(br6, nmse_rnn_512, color ='g', width = barWidth,
        edgecolor ='grey', label ='LSTM-512')


plt.bar(br7, nmse_lstm_128, color ='xkcd:sky blue', width = barWidth,
        edgecolor ='grey', label ='RNN-128')
plt.bar(br8, nmse_lstm_256, color ='tab:blue', width = barWidth,
        edgecolor ='grey', label ='RNN-256')
plt.bar(br9, nmse_lstm_512, color ='b', width = barWidth,
        edgecolor ='grey', label ='RNN-512')
 
# Adding Xticks
plt.xlabel('Horizon', fontsize = 12)
plt.ylabel('RMSE after Compressed Channel Prediction', fontsize = 12)
plt.xticks([r + 4*barWidth for r in range(len(horis))],
        ['3', '5', '7', '9'])
# plt.ylim(-4, 0)
plt.legend()
plt.show()

In [27]:
import matplotlib.pyplot as plt
import numpy as np
nmse_stem_512 = [-3.4409, -3.0037, -2.5981, -2.3735]
nmse_stem_256 = [-3.2731, -2.8746, -2.5058, -2.3395]
nmse_stem_128 = [-3.0267, -2.7942, -2.4437, -2.2798]

nmse_lstm_128 = [-1.0502, -1.0277, -0.8654, -0.8881]
nmse_lstm_256 = [-0.9385, -0.9603, -0.8015, -0.8196]
nmse_lstm_512 = [ -0.8787, -0.8638, -0.7598, -0.7450]

nmse_rnn_128 = [-1.1020, -1.1766, -0.9629, -0.9752]
nmse_rnn_256 = [-0.9735, -1.0429, -0.9191, -0.8518]
nmse_rnn_512 = [-1.0114, -1.007, -0.8056, -0.7980]

#rmse_tran_128 = [22.2789484262, 21.2054104208, 22.9343266487, 22.8131954073]
#rmse_tran_256 = [21.0762034058, 21.9603829382, 22.6214694383, 22.7122474310]
#rmse_tran_512 = [20.166, 21.7080699205, 21.9851315617, 22.3868720531]

barWidth = 0.07
fig = plt.subplots(figsize =(12, 8))
horis = [3, 5, 7, 9]

br1 = np.arange(len(horis))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
br4 = [x + barWidth for x in br3]
br5 = [x + barWidth for x in br4]
br6 = [x + barWidth for x in br5]
br7 = [x + barWidth for x in br6]
br8 = [x + barWidth for x in br7]
br9 = [x + barWidth for x in br8]
#br10 = [x + barWidth for x in br9]
#br11 = [x + barWidth for x in br10]
#br12 = [x + barWidth for x in br11]

plt.bar(br1, rmse_stem_128, color ='y', width = barWidth,
        edgecolor ='grey', label ='STEM-128')
plt.bar(br2, rmse_stem_256, color ='tab:orange', width = barWidth,
        edgecolor ='grey', label ='STEM-256')
plt.bar(br3, rmse_stem_512, color ='r', width = barWidth,
        edgecolor ='grey', label ='STEM-512')

#plt.bar(br4, rmse_tran_128, color ='tab:pink', width = barWidth,
#        edgecolor ='grey', label ='TRANSFORMER-512')
#plt.bar(br5, rmse_tran_256, color ='m', width = barWidth,
#        edgecolor ='grey', label ='TRANSFORMER-256')
#plt.bar(br6, rmse_tran_512, color ='darkviolet', width = barWidth,
#        edgecolor ='grey', label ='TRANSFORMER-128')

plt.bar(br4, rmse_rnn_128, color ='mediumseagreen', width = barWidth,
        edgecolor ='grey', label ='LSTM-128')
plt.bar(br5, rmse_rnn_256, color ='tab:green', width = barWidth,
        edgecolor ='grey', label ='LSTM-256')
plt.bar(br6, rmse_rnn_512, color ='g', width = barWidth,
        edgecolor ='grey', label ='LSTM-512')


plt.bar(br7, rmse_lstm_128, color ='xkcd:sky blue', width = barWidth,
        edgecolor ='grey', label ='RNN-128')
plt.bar(br8, rmse_lstm_256, color ='tab:blue', width = barWidth,
        edgecolor ='grey', label ='RNN-256')
plt.bar(br9, rmse_lstm_512, color ='b', width = barWidth,
        edgecolor ='grey', label ='RNN-512')
 
# Adding Xticks
plt.xlabel('Horizon', fontsize = 12)
plt.ylabel('RMSE after Compressed Channel Prediction', fontsize = 12)
plt.xticks([r + 4*barWidth for r in range(len(horis))],
        ['3', '5', '7', '9'])
# plt.ylim(-4, 0)
plt.legend()
plt.show()

In [25]:
import matplotlib.pyplot as plt
import pickle
train_loss_stem = [0.2698, 0.0473, 0.0117, 0.0086, 0.0059, 0.0053, 0.0047, 0.0043, 0.0040, 0.0034, 0.0033, 0.0031, 0.0030, 0.0029, 0.0027, 0.0026, 0.0026, 0.0025, 0.0025, 0.0024, 0.0024, 0.0023, 0.0023, 0.0023, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021, 0.0021]
train_rmse_stem = [0.2983, 0.1806, 0.1396, 0.1140, 0.1030, 0.0893, 0.0834, 0.0800, 0.0763, 0.0759, 0.0712, 0.0690, 0.0672, 0.0654, 0.0651, 0.0634, 0.0634, 0.0621, 0.0612, 0.0614, 0.0605, 0.0600, 0.0597, 0.0593, 0.0591, 0.0590, 0.0586, 0.0585, 0.0584, 0.0581, 0.0580, 0.0579, 0.0579, 0.0577, 0.0576, 0.0575, 0.0575, 0.0575, 0.0574, 0.0573, 0.0573]
# 0.1030 = 0.1230
# plt.plot(train_rmse_stem)
# plt.plot(avg_loss[0:40])
# plt.xlabel('epochs',fontweight ='bold', fontsize = 12)
# plt.ylabel('RMSE loss during training',fontweight ='bold', fontsize = 12)
# plt.show()

#avg_loss_rnn = [52.9602, 50.9641, 49.3253, 47.8729, 46.5499, 45.3367, 44.2122, 43.1625, 42.1598, 41.19, 40.2551, 39.3397, 38.4583, 37.5905, 36.7461, 35.9289, 35.134, 34.3678, 33.6181, 32.9036, 32.2012, 31.5277, 30.8788, 30.2446, 29.63, 29.0384, 28.4532, 27.8978, 27.3725, 26.8557, 26.3585, 25.8909, 25.4297, 24.9941, 24.5849, 24.1615, 23.7443, 23.3636, 22.9839, 22.6152]
#avg_loss_rnn = [55.9602, 53.9641, 52.3253, 50.8729, 49.5499, 48.3367, 47.2122, 46.1625, 45.1598, 44.19, 43.2551, 42.3397, 41.4583, 40.5905, 39.7461, 38.9289, 38.134, 37.3678, 36.6181, 35.9036, 35.2012, 34.5277, 33.8788, 33.2446, 32.63, 32.0384, 31.4532, 30.8978, 30.3725, 29.8557, 29.3585, 28.8909, 28.4297, 27.9941, 27.5849, 27.1615, 26.7443, 26.3636, 25.9839, 25.6152]
#avg_loss_rnn = [52.6144, 50.3449, 48.5387, 46.9309, 45.4993, 44.2113, 43.0646, 42.0646, 41.2036, 40.4097, 39.6939, 39.0199, 38.4353, 37.8201, 37.2265, 36.6126, 36.0013, 35.3849, 34.7866, 34.1825, 33.5449, 32.8498, 32.1366, 31.4401, 30.7984, 30.2065, 29.6689, 29.1778, 28.6956, 28.2404, 27.7963, 27.3437, 26.8871, 26.4566, 26.0290, 25.6221, 25.1925, 24.7500, 24.3100, 23.8482]
#avg_loss_rnn = [55.1942, 53.347, 51.8752, 50.5996, 49.4387, 48.3563, 47.3359, 46.3719, 45.4445, 44.5572, 43.7021, 42.8745, 42.0737, 41.29, 40.5361, 39.8076, 39.0928, 38.4013, 37.7361, 37.0911, 36.4632, 35.8463, 35.2478, 34.656, 34.084, 33.5249, 32.9703, 32.4235, 31.8927, 31.36, 30.8589, 30.3508, 29.867, 29.3875, 28.9084, 28.4554, 28.0162, 27.5824, 27.1481, 26.7298]
avg_loss_rnn = [55.9236, 53.5858, 51.8187, 50.3261, 49.0433, 47.7919, 46.5947, 45.5121, 44.461, 43.519, 42.611, 41.6953, 40.8399, 39.9977, 39.1825, 38.3657, 37.5643, 36.7996, 36.0567, 35.3813, 34.7616, 34.1597, 33.6267, 33.114, 32.6331, 32.1935, 31.7584, 31.2979, 30.8197, 30.3658, 29.8969, 29.4674, 28.9864, 28.5255, 28.0818, 27.6133, 27.1283, 26.657, 26.2557, 25.8471]
#avg_loss_lstm =  [53.5067, 52.1926, 50.8675, 49.6986, 48.7009, 47.827, 47.001, 46.1704, 45.3692, 44.5596, 43.7978, 43.0583, 42.3486, 41.6699, 41.028, 40.3998, 39.789, 39.1931, 38.5858, 38.0086, 37.4243, 36.8492, 36.2855, 35.7374, 35.1709, 34.6299, 34.0972, 33.5962, 33.1131, 32.6421, 32.1405, 31.674, 31.2197, 30.7978, 30.4051, 29.9945, 29.5988, 29.1876, 28.8158, 28.4541]
avg_loss_lstm = [55.5067, 54.1926, 52.8675, 51.6986, 50.7009, 49.827, 49.001, 48.1704, 47.3692, 46.5596, 45.7978, 45.0583, 44.3486, 43.6699, 43.028, 42.3998, 41.789, 41.1931, 40.5858, 40.0086, 39.4243, 38.8492, 38.2855, 37.7374, 37.1709, 36.6299, 36.0972, 35.5962, 35.1131, 34.6421, 34.1405, 33.674, 33.2197, 32.7978, 32.4051, 31.9945, 31.5988, 31.1876, 30.8158, 30.4541]

fig = plt.figure()
ax = fig.add_subplot(111)
lns1 = ax.plot(train_rmse_stem, '-', label = 'STEM', color ='y', linewidth=2)
ax2 = ax.twinx()
lns2 = ax2.plot(avg_loss_rnn[0:40], '-', label = 'LSTM', color ='mediumseagreen', linewidth=2)
lns3 = ax2.plot(avg_loss_lstm[0:40], '-', label = 'RNN', color ='xkcd:sky blue', linewidth=2)
#lns4 = ax2.plot(avg_loss_tran[0:40], '-', label = 'Transformer', color ='tab:pink', linewidth=2)

ax.grid()
ax.set_xlabel('epochs', fontsize = 12)
ax.set_ylabel('RMSE loss during training for STEM', fontsize = 12)
ax2.set_ylabel('RMSE loss during training for Transformer/LSTM/RNN', fontsize = 12)
lns = lns1+lns2+lns3
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc=0)
plt.show()